In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import sleep
import itertools
import cv2
import os
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display

We are going to try MobileNet for regression task

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import sleep
import itertools
import cv2
import os
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Define a useful function
def get_image(f_path):
    '''
    Returns the image from a path
    '''
    img_labs = ['jpg','png']
    if any(x in img_labs for x in f_path.split('.')):
        file = os.path.join(folder,f_path)
        image = open(file,'rb').read()
        return image
    
# Do the actual work here
folder = '../input/petfinder-pawpularity-score/train'
files  = os.listdir(folder)
sample_files = files[0:18]

### Below we see some sample files

In [ ]:
images = [get_image(x) for x in sample_files]
children = [widgets.Image(value = img) for img in images if str(type(img)) != '<class \'NoneType\'>']
labels = ['{}'.format(i) for i in range(len(children))]

# Customize your layout here:
box_layout = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='50%')

# Create the widget
tab = widgets.Tab()
tab.children = children

# Label em'!
for i in range(len(children)):
    tab.set_title(i,labels[i])

display(tab)

In [ ]:
import pandas as pd
df_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df_train_below_50 = df_train.loc[(df_train['Pawpularity'] < 50)]
df_train_grt_50 = df_train.loc[(df_train['Pawpularity'] >= 50)]


In [ ]:
#Generating file name with proper extension
from tensorflow.keras.preprocessing.image import ImageDataGenerator

df_train_below_50['FileName'] = df_train_below_50['Id']+".jpg"

def augmentData(df,img_width,img_height,x_col,y_col,batch_size):
    datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True,
                                   fill_mode = "nearest", zoom_range = 0.2,
                                   width_shift_range = 0.2, height_shift_range=0.2,
                                   rotation_range=30,validation_split=0.20)
    
    #Creating training generator and validation generator
    train_generator = datagen.flow_from_dataframe(dataframe=df, directory="../input/petfinder-pawpularity-score/train/", 
                                              x_col=x_col, y_col=y_col, 
                                              class_mode="raw", target_size=(img_width, img_height), 
                                              batch_size=batch_size,shuffle=True)
    
    validation_generator = datagen.flow_from_dataframe(dataframe=df, directory="../input/petfinder-pawpularity-score/train/", 
                                              x_col=x_col, y_col=y_col, 
                                              class_mode="raw", target_size=(img_width, img_height), 
                                              batch_size=batch_size,subset='validation',shuffle=True)
    return train_generator,validation_generator

#We are using the function to augment and generate and each time you call the function it is going to shuffle
#the data for generator which can also used for cross validation
train_generator,validation_generator = augmentData(df_train_below_50,100,100,"FileName","Pawpularity",64)


### Let us view some sample Images which are augmented
This will help us understand the variety of images

In [ ]:
# generate samples and plot
from matplotlib import pyplot
batch=next(train_generator)  # returns the next batch of images and labels 
print(batch[0].shape) # batch[0] is the images, batch[1] are the labels
img=batch[0][0]   # this is the first image  batch[0][1] would be the next image
print (img.shape)
plt.imshow(img) 

https://www.kaggle.com/gbalachandhiran/mobilenetmorenet?scriptVersionId=78395874&cellId=13

###Depthwise convolution is the channel-wise DK×DK spatial convolution. Suppose in the figure above, we have 5 channels, then we will have 5 DK×DK spatial convolution.

Pointwise convolution actually is the 1×1 convolution to change the dimension.

In [ ]:
#We are going to use resnet for transfer learning and for regression task
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
#Construct resnet model
def mobileNetV2(height,width,fc_layer_nodes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(height,width,3),
                                               include_top=False,
                                               weights='imagenet')
    #Freeze weights
    for layer in base_model.layers:
        layer.trainable = False
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(fc_layer_nodes, activation='relu')(x)
    prediction = layers.Dense(1)(x)
    mobNet_v2_regression = Model(inputs = base_model.input, outputs = prediction)
    return mobNet_v2_regression

mobNet_v2_regression = mobileNetV2(160,160,1000)

### We will write a custom function to calculate RMSE

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        y_true = tf.cast(y_true, dtype='float64')
        y_pred = tf.cast(y_pred, dtype='float64')
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [ ]:
mobNet_v2_regression_below_50 = mobileNetV2(160,160,500)
mobNet_v2_regression_below_50.compile(optimizer='adam', loss=root_mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError(
    name="root_mean_squared_error", dtype=None)])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#Since resnet will accept 160*160 in top layer hence we need to adjust the generators
train_generator,validation_generator = augmentData(df_train_below_50,160,160,"FileName","Pawpularity",64)

history = mobNet_v2_regression_below_50.fit(train_generator,steps_per_epoch=100,epochs=10,validation_data= validation_generator,callbacks=[callback])

In [ ]:

plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model RMSE')
plt.ylabel('RMSE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### We will now build another model which will help us regress for pawpularity score greater than 50

In [ ]:
#Generating file name with proper extension
from tensorflow.keras.preprocessing.image import ImageDataGenerator

df_train_grt_50['FileName'] = df_train_grt_50['Id']+".jpg"

def augmentData(df,img_width,img_height,x_col,y_col,batch_size):
    datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True,
                                   fill_mode = "nearest", zoom_range = 0.2,
                                   width_shift_range = 0.2, height_shift_range=0.2,
                                   rotation_range=30,validation_split=0.20)
    
    #Creating training generator and validation generator
    train_generator = datagen.flow_from_dataframe(dataframe=df, directory="../input/petfinder-pawpularity-score/train/", 
                                              x_col=x_col, y_col=y_col, 
                                              class_mode="raw", target_size=(img_width, img_height), 
                                              batch_size=batch_size,shuffle=True)
    
    validation_generator = datagen.flow_from_dataframe(dataframe=df, directory="../input/petfinder-pawpularity-score/train/", 
                                              x_col=x_col, y_col=y_col, 
                                              class_mode="raw", target_size=(img_width, img_height), 
                                              batch_size=batch_size,subset='validation',shuffle=True)
    return train_generator,validation_generator

#We are using the function to augment and generate and each time you call the function it is going to shuffle
#the data for generator which can also used for cross validation
train_generator,validation_generator = augmentData(df_train_grt_50,100,100,"FileName","Pawpularity",64)

In [ ]:
mobNet_v2_regression_grt_50 = mobileNetV2(160,160,500)
mobNet_v2_regression_grt_50.compile(optimizer='adam', loss=root_mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError(
    name="root_mean_squared_error", dtype=None)])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', patience=10)
#Since resnet will accept 160*160 in top layer hence we need to adjust the generators
train_generator,validation_generator = augmentData(df_train_grt_50,160,160,"FileName","Pawpularity",64)

history = mobNet_v2_regression_grt_50.fit(train_generator,steps_per_epoch=10,epochs=20,validation_data= validation_generator,callbacks=[callback])

In [ ]:
# serialize model to JSON
model_json = mobNet_v2_regression_below_50.to_json()
with open("mobNet_v2_regression_below_50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
mobNet_v2_regression.save_weights("mobNet_v2_regression_below_50.h5")
print("Saved model to disk")

In [ ]:
# serialize model to JSON
model_json = mobNet_v2_regression_grt_50.to_json()
with open("mobNet_v2_regression_grt_50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
mobNet_v2_regression.save_weights("mobNet_v2_regression_grt_50.h5")
print("Saved model to disk")